In [19]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv("C:/Users/test/Downloads/olympix_data_organized_with_header (2).csv")

In [21]:
db_connection_str = "mysql+pymysql://root:@127.0.0.1:3306/olympic"
db_connection = create_engine(db_connection_str)

In [22]:
df.to_sql(name='master_table', con=db_connection, if_exists='replace', index=False)

In [23]:
df.head()

,name,age,country,year,Date_Given,sports,gold_medal,silver_medal,bronze_medal,total_medal
0,Michael Phelps,23,United States,2008,2008-08-24,Swimming,8,0,0,8
1,Michael Phelps,19,United States,2004,2004-08-29,Swimming,6,0,2,8
2,Michael Phelps,27,United States,2012,2012-08-12,Swimming,4,2,0,6
3,Natalie Coughlin,25,United States,2008,2008-08-24,Swimming,1,2,3,6
4,Aleksey Nemov,24,Russia,2000,2000-10-01,Gymnastics,2,1,3,6


In [24]:
# create table players
df_names = df.drop_duplicates(subset=['name'])
#drop every columns except for name
df_names = df_names.drop(columns=['age', 'country', 'Date_Given', 'gold_medal', 'silver_medal', 'sports', 'year',"bronze_medal",'total_medal']).reset_index(drop=True)
#add a new column with the name of player_id
df_names['player_id'] = df_names.index
#add 1 to every player_id
df_names['player_id'] = df_names['player_id'] + 1
#concat "player" wih the player_id
df_names['player_id'] = df_names['player_id'].astype(str)
df_names['player_id'] = "player_" + df_names['player_id']
df_names.head()

,name,player_id
0,Michael Phelps,player_1
1,Natalie Coughlin,player_2
2,Aleksey Nemov,player_3
3,Alicia Coutts,player_4
4,Missy Franklin,player_5


In [25]:
#add this value to olympic dataframe in table players
df_names.to_sql(name='players', con=db_connection, if_exists='replace', index=False)

In [26]:
#country table
df_country = df.drop_duplicates(subset=['country'])
df_country = df_country.drop(columns=['age', 'name', 'Date_Given', 'gold_medal', 'silver_medal', 'sports', 'year',"bronze_medal",'total_medal']).reset_index(drop=True)
df_country['country_id'] = df_country.index
df_country['country_id'] = df_country['country_id'] + 1
df_country['country_id'] = df_country['country_id'].astype(str)
df_country['country_id'] = "country_" + df_country['country_id']
df_country.head()

,country,country_id
0,United States,country_1
1,Russia,country_2
2,Australia,country_3
3,Canada,country_4
4,Norway,country_5


In [27]:
#add this value to olympic dataframe in table players
df_country.to_sql(name='country', con=db_connection, if_exists='replace', index=False)

In [28]:
#sports table
df_sports = df.drop_duplicates(subset=['sports'])
df_sports = df_sports.drop(columns=['age', 'name', 'country', 'Date_Given', 'gold_medal', 'silver_medal', 'year',"bronze_medal",'total_medal']).reset_index(drop=True)
df_sports['sports_id'] = df_sports.index
df_sports['sports_id'] = df_sports['sports_id'] + 1
df_sports['sports_id'] = df_sports['sports_id'].astype(str)
df_sports['sports_id'] = "sports_" + df_sports['sports_id']
df_sports.head()

,sports,sports_id
0,Swimming,sports_1
1,Gymnastics,sports_2
2,Speed Skating,sports_3
3,Cross Country Skiing,sports_4
4,Short-Track Speed Skating,sports_5


In [29]:
#add this value to olympic dataframe in table players
df_sports.to_sql(name='sport', con=db_connection, if_exists='replace', index=False)

In [32]:
#create table sports_player
df_sports_player = df.drop_duplicates(subset=['name'])
df_sports_player = df_sports_player.drop(columns=['age', 'Date_Given', 'gold_medal', 'silver_medal', 'year',"bronze_medal",'total_medal']).reset_index(drop=True)
df_sports_player = pd.merge(df_sports_player, df_names, on='name')
df_sports_player = pd.merge(df_sports_player, df_sports, on='sports')
df_sports_player = pd.merge(df_sports_player, df_country, on='country')
df_sports_player = df_sports_player.drop(columns=['name', 'sports','country'])
df_sports_player.head()

,player_id,sports_id,country_id
0,player_1,sports_1,country_1
1,player_2,sports_1,country_1
2,player_5,sports_1,country_1
3,player_6,sports_1,country_1
4,player_7,sports_1,country_1


In [33]:
#add this value to olympic dataframe in table sports_player
df_sports_player.to_sql(name='join_table', con=db_connection, if_exists='replace', index=False)

In [34]:
#create table medals
df_medals = df.copy()
df_medals = df_medals.drop(columns=['sports', 'country']).reset_index(drop=True)
df_medals = pd.merge(df_medals, df_names, on='name')
#drop name
df_medals = df_medals.drop(columns=['name'])
df_medals.head()

,age,year,Date_Given,gold_medal,silver_medal,bronze_medal,total_medal,player_id
0,23,2008,2008-08-24,8,0,0,8,player_1
1,19,2004,2004-08-29,6,0,2,8,player_1
2,27,2012,2012-08-12,4,2,0,6,player_1
3,25,2008,2008-08-24,1,2,3,6,player_2
4,21,2004,2004-08-29,2,2,1,5,player_2


In [35]:
#change column postion from left to right
df_medals = df_medals[['player_id','age','year','Date_Given','gold_medal', 'silver_medal', 'bronze_medal', 'total_medal']]
df_medals.head()

,player_id,age,year,Date_Given,gold_medal,silver_medal,bronze_medal,total_medal
0,player_1,23,2008,2008-08-24,8,0,0,8
1,player_1,19,2004,2004-08-29,6,0,2,8
2,player_1,27,2012,2012-08-12,4,2,0,6
3,player_2,25,2008,2008-08-24,1,2,3,6
4,player_2,21,2004,2004-08-29,2,2,1,5


In [36]:
#add this value to olympic dataframe in table sports_player
df_medals.to_sql(name='medal', con=db_connection, if_exists='replace', index=False)